In [ ]:
from sdgym import load_dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from sklearn.model_selection import train_test_split

In [ ]:
import xgboost as xgb

In [ ]:
%matplotlib inline

In [ ]:
data = load_dataset('alarm')

In [ ]:
#data, categorical_columns, ordinal_columns = load_dataset('alarm')

In [ ]:
data

In [ ]:
data['tables']

In [ ]:
data

In [ ]:
import pgmpy

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch

In [ ]:
from pgmpy.estimators import HillClimbSearch, BicScore, ExhaustiveSearch

In [ ]:
df = pd.DataFrame(data)
df.columns = [str(i) for i in df.columns]

In [ ]:

# learn graph structure 
est = TreeSearch(df, root_node=df.columns[0])
dag = est.estimate(estimator_type="tan", class_node='1')

In [ ]:

# alternative graph structure 
est2 = TreeSearch(df, root_node=df.columns[0])
dag2 = est2.estimate(estimator_type="chow-liu")

In [ ]:
est = HillClimbSearch(df)

In [ ]:
best_model = est.estimate() # start_dag=dag)

In [ ]:
nx.draw(best_model, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

In [ ]:
edges = best_model.edges()

In [ ]:
edges

In [ ]:
from pgmpy.estimators import BayesianEstimator

# there are many choices of parametrization, here is one example
model = BayesianModel(best_model.edges())
model.fit(df, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=0.1)


In [ ]:
print(model.get_cpds('2'))

In [ ]:
# set up train-test sample.
# the test sample is used to calibrate the output of the classifier

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(data, np.ones(data.shape[0]), test_size=0.35,
                                                        random_state=0)



In [ ]:
X1_train.shape

In [ ]:
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.svm import SVC

In [ ]:
clf=MLPClassifier(random_state=0, max_iter=1000, early_stopping=True)

In [ ]:
clf = xgb.XGBClassifier(
    n_estimators=250,
    reg_lambda=1,
    gamma=0,
    max_depth=9
)

In [ ]:
import inspect

In [ ]:
argspecs = inspect.getfullargspec(clf.fit)

In [ ]:
support_weight = 'sample_weight' in argspecs.args

In [ ]:
n_one = len(X1_train)
n_zero = n_one

In [ ]:
from pgmpy.sampling import BayesianModelSampling

# sample data from BN
inference = BayesianModelSampling(model)
df_data = inference.forward_sample(size=n_zero, return_type='dataframe', seed=0)

df_data.columns = [int(c) for c in df_data.columns]

X0_train = df_data[sorted(df_data.columns)].values

In [ ]:
zeros = np.zeros(n_zero)
ones = np.ones(n_one)

yy = np.concatenate([zeros, ones], axis = 0)
XX = np.concatenate([X0_train, X1_train], axis = 0)

In [ ]:
clf = clf.fit(XX, yy)

In [ ]:
p0 = clf.predict_proba(X0_train)[:, 1]
p2 = clf.predict_proba(X1_train)[:, 1]


In [ ]:
nbins = 100
plt.figure(figsize=(12,7))
plt.hist(p0, bins=100, range=(0,1), alpha=0.5, log=True, density=True)
plt.hist(p2, bins=100, range=(0,1), alpha=0.5, log=True, density=True)


In [ ]:
# calibrate the probabilities, using the test sample and a new null sample

In [ ]:
df_data = inference.forward_sample(size=100000, return_type='dataframe', seed=10)

df_data.columns = [int(c) for c in df_data.columns]

X = df_data[sorted(df_data.columns)].values

In [ ]:
p0 = clf.predict_proba(X)[:, 1]
p2 = clf.predict_proba(X1_test)[:, 1]


In [ ]:
nbins = 100
plt.figure(figsize=(12,7))
plt.hist(p0, bins=100, range=(0,1), alpha=0.5, log=True, density=True)
plt.hist(p2, bins=100, range=(0,1), alpha=0.5, log=True, density=True)


In [ ]:
0.5 / np.power(3500, 1/3.)

In [ ]:
nbins = 100
binning = np.linspace(0, 1, nbins+1)

hist_p0, bin_edges = np.histogram(p0, binning)
hist_p1, bin_edges = np.histogram(p2, binning)


hist_p0

hist_p1

def poisson_uncertainty(n):
    sigman = np.sqrt(n)
    # correct poisson counts of zero.
    sigman[sigman == 0] = 1.
    return sigman

def fraction_and_uncertainty(a, b, sigma_a, sigma_b):
    frac_a = a / (a + b)
    frac_b = b / (a + b)
    sigma_fa2 = np.power(frac_b * sigma_a, 2) / np.power(a + b, 2)  +  np.power(frac_a * sigma_b, 2) / np.power(a + b, 2)
    return frac_a, np.sqrt(sigma_fa2)

rest_p0 = np.sum(hist_p0) - hist_p0
rest_p1 = np.sum(hist_p1) - hist_p1

sigma_bin0 = poisson_uncertainty(hist_p0)
sigma_rest0 = poisson_uncertainty(rest_p0)

sigma_bin1 = poisson_uncertainty(hist_p1)
sigma_rest1 = poisson_uncertainty(rest_p1)

frac0, sigma_frac0 = fraction_and_uncertainty(hist_p0, rest_p0, sigma_bin0, sigma_rest0)
frac1, sigma_frac1 = fraction_and_uncertainty(hist_p1, rest_p1, sigma_bin1, sigma_rest1)

p1calib, sigma_p1calib = fraction_and_uncertainty(frac1, frac0, sigma_frac1, sigma_frac0)

sample_weight = 1 / (sigma_p1calib * sigma_p1calib)

min(sample_weight)

sample_weight /= min(sample_weight)

sample_weight

In [ ]:
from sklearn.isotonic import IsotonicRegression
from scipy import interpolate

In [ ]:
# we recalibrate per probability bin. NO interpolation (not valid in highest bin)

hist_p0, bin_edges = np.histogram(p0, bins=nbins, range=(0, 1))
hist_p1, bin_edges = np.histogram(p2, bins=nbins, range=(0, 1)) #### !!!! p2
bin_centers = bin_edges[:-1] + 0.5/nbins

hnorm_p0 = hist_p0 / sum(hist_p0)
hnorm_p1 = hist_p1 / sum(hist_p1)
hnorm_sum = hnorm_p0 + hnorm_p1
p1cb = np.divide(hnorm_p1, hnorm_sum, out=np.zeros_like(hnorm_p1), where=hnorm_sum != 0)
# self.p1cb = p1cb, bin_centers

# use isotonic regression to smooth out potential fluctuations in the p1 values
# isotonic regression assumes that p1 can only be a rising function.
# I’m assuming that if a classifier predicts a higher probability, the calibrated probability
# will also be higher. This may not always be right, but I think generally it is a safe one.
iso_reg = IsotonicRegression(y_min=0, y_max=1).fit(bin_centers, p1calib, sample_weight)
p1pred = iso_reg.predict(bin_centers)

p1f_ = interpolate.interp1d(bin_edges[:-1], p1pred, kind='previous', bounds_error=False, fill_value="extrapolate")



In [ ]:
p1lin = p1f_(bin_centers)

In [ ]:
plt.figure(figsize=(12,7))
#plt.plot(bin_centers, p1cb)
plt.plot(bin_centers, p1cb)
plt.plot(bin_centers, bin_centers)
plt.plot(bin_centers, p1lin)
#plt.plot(bin_centers, p2lin)

In [ ]:
x = np.linspace(0.95,1,500)

In [ ]:
pp = p1f_(x)

In [ ]:
plt.figure(figsize=(12,7))
#plt.plot(bin_centers, p1cb)
plt.plot(x, pp)

In [ ]:
maxp1 = p1f_(0.999)

In [ ]:
max_weight = maxp1 / (1. - maxp1)
max_weight

In [ ]:
# validation - part 1: check if reweighting works okay

In [ ]:
from pgmpy.sampling import BayesianModelSampling

# sample data from BN
inference = BayesianModelSampling(model)

df_data = inference.forward_sample(size=250000, return_type='dataframe', seed=1)

df_data.columns = [int(c) for c in df_data.columns]

X = df_data[sorted(df_data.columns)].values

In [ ]:
p0 = clf.predict_proba(X)[:, 1]
nominator = p1f_(p0)
denominator = 1 - nominator
weight = np.divide(nominator, denominator, out=np.ones_like(nominator), where=denominator != 0)

In [ ]:
len(X), sum(weight)

In [ ]:
np.sqrt(np.sum(weight * weight))

In [ ]:
plt.hist(weight[weight < 10], bins=100, log=True)

In [ ]:
max_weight

In [ ]:
p0 = clf.predict_proba(X)[:, 1]
nominator = p1f_(p0)
denominator = 1 - nominator
weight = np.divide(nominator, denominator, out=np.ones_like(nominator), where=denominator != 0)

In [ ]:
from random import choices

In [ ]:
#data, sample_weights = self._sample_no_transform(n_samples, random_state)
pop = np.asarray(range(X.shape[0]))
probs = weight/np.sum(weight)
sample = choices(pop, probs, k=X.shape[0])
Xtrans = X[sample]


In [ ]:
p0 = clf.predict_proba(Xtrans)[:, 1]
p2 = clf.predict_proba(X1_test)[:, 1]


In [ ]:
plt.figure(figsize=(12,7))
plt.hist(p0, bins=100, range=(0,1), alpha=0.5, density=True, log=True) #, weights=weight)#, log=True)
plt.hist(p2, bins=100, range=(0,1), alpha=0.5, density=True)


In [ ]:
# validation - part 2: plot distributions

In [ ]:
i = 1
plt.figure(figsize=(12,7))
plt.hist(X[:, i], bins=100, range=(0,1), alpha=0.5, density=True)#, log=True)
plt.hist(X1_test[:, i], bins=100, range=(0,1), alpha=0.5, density=True)


In [ ]:
# validation part 3: check number of duplicates

In [ ]:
df_data = inference.forward_sample(size=500000, return_type='dataframe', seed=2)
df_data.columns = [int(c) for c in df_data.columns]
X10k = df_data[sorted(df_data.columns)].values

In [ ]:
p0 = clf.predict_proba(X10k)[:, 1]
nominator = p1f_(p0)
denominator = 1 - nominator
weight = np.divide(nominator, denominator, out=np.ones_like(nominator), where=denominator != 0)

In [ ]:
sum(weight)

In [ ]:
np.sqrt(np.sum(weight * weight))

In [ ]:
uo, co = np.unique(X10k, axis=0, return_counts=True)
countso = np.sort(co)[::-1] / 50

In [ ]:
pop = np.asarray(range(X10k.shape[0]))
probs = weight/np.sum(weight)
sample = choices(pop, probs, k=X10k.shape[0])
Xtrans = X10k[sample]


In [ ]:
u, c = np.unique(Xtrans, axis=0, return_counts=True)

In [ ]:
counts = np.sort(c)[::-1] / 50

In [ ]:
u, c = np.unique(data, axis=0, return_counts=True)

In [ ]:
c2 = np.sort(c)[::-1] 

In [ ]:
plt.figure(figsize=(12,7))
plt.bar(list(range(40)), c2[:40], alpha=0.3)
plt.bar(list(range(40)), counts[:40], alpha=0.3)
plt.bar(list(range(40)), countso[:40], alpha=0.3)

In [ ]:
plt.figure(figsize=(12,7))
plt.bar(list(range(40)), c2[:40], alpha=0.3)
plt.bar(list(range(40)), counts[:40], alpha=0.3)

In [ ]:
plt.figure(figsize=(12,7))
plt.bar(list(range(40)), c2[:40], alpha=0.3)
plt.bar(list(range(40)), counts[:40], alpha=0.3)

In [ ]:
import numpy as np
import pandas as pd
from sdgym import benchmark
from sdgym import load_dataset
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf
from synthsonic.models.categorical_utils import categorical_round, vec_translate, categorical_frequency_mapping, \
            categorical_frequency_inverse_mapping, encode_one_hot, decode_one_hot
from timeit import default_timer as timer
import xgboost as xgb
from sklearn.decomposition import PCA
%matplotlib inline

from functools import partial

In [ ]:

df = pd.DataFrame(Xtrans)
df.to_csv('test.csv', index=False)


In [ ]:
def KDECopulaNNPdf_RoundCategorical(real_data, categorical_columns, ordinal_columns, times=None):
    df = pd.read_csv('test.csv')
    data = df.values[:25000]
    return data

In [ ]:
alarm_times = []
alarm_thing = partial(KDECopulaNNPdf_RoundCategorical, times=alarm_times)
alarm_thing.__name__ = KDECopulaNNPdf_RoundCategorical.__name__


In [ ]:
alarm_scores = benchmark(synthesizers=[alarm_thing], datasets=['alarm'])

In [ ]:
alarm_scores

In [ ]:
if False:
    alarm_scores = benchmark(synthesizers=[alarm_thing], datasets=['alarm'])
    alarm_scores.drop(columns=['timestamp'], inplace=True)
    exec_time = ['N/A'] * 9 + [round(np.mean(alarm_times), 2)]
    alarm_scores['alarm/exec_time(s)'] = exec_time    